train_data.csv中的数据点（包含实际坐标和分类）作为训练数据，来预测四个波段的像素点的类别。

In [ ]:
import csv
import os
import numpy as np
from sklearn import tree                       
from osgeo import gdal
import ospybook as pb
folder = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Landsat\Utah'
raster_fns = ['LE70380322000181EDC02_60m.tif', 'LE70380322000181EDC02_TIR_60m.tif'] # 模型训练和预测的数据

train_fn = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Utah\training_data.csv'
gap_fn = r'D:\Projects\VsCode\Python\img_processing_system\osgeopy-data\Utah\landcover60.tif' # 提供colortable
os.chdir(folder)


# Read coordinates and class from training_data.csv
xys = []                                                                 
classes = []                                                            
with open(train_fn) as fp:                                             
    reader = csv.reader(fp)                                            
    next(reader)                                                       
    for row in reader:      
        print(row)                                            
        xys.append([float(n) for n in row[:2]])                         
        classes.append(int(row[2]))  

['377454.1803', '4447158.0107', '82']
['372684.117359', '4443741.95232', '119']
['372822.119284', '4443875.9544', '48']
['371906.125021', '4445339.96952', '48']
['372440.150973', '4447641.9985', '119']
['373894.161207', '4447660.00399', '48']
['375160.183605', '4448996.02492', '48']
['372892.172449', '4449657.02437', '37']
['370233.151958', '4451736.04212', '37']
['374156.187551', '4450188.0356', '48']
['375682.207572', '4450918.05041', '48']
['376904.218328', '4450951.05562', '48']
['378844.23282', '4450777.06109', '48']
['379207.285945', '4454935.11445', '48']
['377751.32288', '4459619.16385', '48']
['381983.274751', '4451922.08808', '76']
['381123.238853', '4449754.05706', '48']
['380478.240428', '4450285.0613', '48']
['380186.208551', '4447938.03046', '48']
['367024.024636', '4441223.9163', '37']
['364255.007423', '4444369.94884', '37']
['365502.033271', '4445593.96383', '124']
['366701.057615', '4446568.97639', '62']
['368029.078406', '4446693.9798', '62']
['382420.242027', '44492

In [2]:
# 此代码块作用：用来理解实际坐标转像素offset，对于分类并无作用。
ds = gdal.Open(raster_fns[0])   
gt = ds.GetGeoTransform()
inv_gt = gdal.InvGeoTransform(gt)
offsets = gdal.ApplyGeoTransform(inv_gt, 372684.117359, 4443741.95232)
print(offsets)
xoff, yoff = map(int, offsets)
value = ds.GetRasterBand(1).ReadAsArray(xoff, yoff, 1, 1)[0,0]


[1321.6519559833341, 2177.884128000005]


In [2]:
# Get pixel offsets
#   Then you open one of the raster datasets so you can create a transformer object to
#   convert between map coordinates and pixel offsets. You use this with your list of coordinates
#   to get pixel offsets in two lists called cols and rows.
ds = gdal.Open(raster_fns[0])                                          
pixel_trans = gdal.Transformer(ds, None, []) # 见Geoprocessing with python 214页                    
offset, ok = pixel_trans.TransformPoints(True, xys) # xys是从training_data.csv中取的。
print("Len of offset = ",len(offset))
print(offset)
cols, rows, z = zip(*offset)  

Len of offset =  578
[(1401.1530050000001, 2120.9498216666543, 0.0), (1321.6519559833341, 2177.884128000005, 0.0), (1323.9519880666667, 2175.650760000004, 0.0), (1308.6854170166662, 2151.2505079999974, 0.0), (1317.5858495499997, 2112.8833583333326, 0.0), (1341.8193534500006, 2112.5832668333314, 0.0), (1362.9197267500003, 2090.3162513333373, 0.0), (1325.1195408166668, 2079.2995938333333, 0.0), (1280.8025326333327, 2044.6492979999894, 0.0), (1346.1864591833328, 2070.4494066666666, 0.0), (1371.6201261999995, 2058.282493166669, 0.0), (1391.9869721333334, 2057.7324063333363, 0.0), (1424.3205469999994, 2060.632315166673, 0.0), (1430.3714324166667, 1991.3314258333266, 0.0), (1406.1053813333328, 1913.2639358333254, 0.0), (1476.6379125166668, 2041.5485320000007, 0.0), (1462.3039808833328, 2077.68238233334, 0.0), (1451.5540071333335, 2068.832311666658, 0.0), (1446.686809183333, 2107.9494923333405, 0.0), (1227.3170772666663, 2219.851395000005, 0.0), (1181.1667903833331, 2167.417519333336, 0.0), (

In [ ]:
data = pb.stack_bands(raster_fns)
print("data.shape= ", data.shape)
# Convert rows and cols to integer numpy arrays for indexing
rows_int = np.array(rows).astype(int)
cols_int = np.array(cols).astype(int)
# Sample satellite data at pixel offsets 
sample = data[rows_int, cols_int, :] # 模型的训练数据,四个band相当于四个特征。data由4个band对应二维数组组成，二维数组中每个值代表一个像素值。取四个band在rows_int, cols_int处的像素值
'''
This samples the 3D array at each of the provided row and column offsets, and returns every value 
in the third dimension, which is the four different satellite bands. 
The result is a two-dimensional array, where each row contains the four pixel values from the four bands.
'''

data.shape=  (3631, 3996, 4)


TypeError: len() of unsized object

In [ ]:
print(data.shape) 
# 问题：明明只有四个band，也就是4个二维数组，为什么这里有3631个？

# 答：
# data.shape 输出的是 (3631, 3996, 4)，其中 3631 是行数（rows），3996 是列数（cols），4 是波段数（bands）。
# 这里的 3631 不是 band 数，而是影像的像素行数。每个 band 都是一个 3631x3996 的二维数组，四个 band 叠加后就是一个三维数组。
# 所以 data.shape[0] 是行数，不是 band 数。band 数在 data.shape[2]。

# 关于你的问题：
# arr_3D = [arr_2d_1, arr_2d_2, arr_2d_3, arr_2d_4] 这样构造的数组，np.array(arr_3D) 的 shape 是 (4, 2, 3)，
# 其中 4 是二维数组的个数（即 band 数），2 是每个二维数组的行数，3 是每个二维数组的列数。
# 这和影像数据的存储方式有关，通常遥感影像是 shape=(rows, cols, bands)，
# 而你举的例子是 shape=(bands, rows, cols)。两者没有矛盾，只是数据组织方式不同。
# stack_bands 函数会把 band 维度放在最后，变成 (rows, cols, bands)，
# 而你直接用 list 构造则是 (bands, rows, cols)。
print(len(data))
print("sample.shape=", sample.shape)
print(sample) 

(3631, 3996, 4)
3631
(578, 4)
[[135 114  91 235]
 [156 128 112 254]
 [151 128 110 241]
 ...
 [146 119 104 243]
 [146 120 105 233]
 [152 121 101 250]]


# 训练模型

In [6]:
# 决策树模型
"""
# Fit the classification tree
clf = tree.DecisionTreeClassifier(max_depth=5)                           
clf = clf.fit(sample, classes)
"""

'\n# Fit the classification tree\nclf = tree.DecisionTreeClassifier(max_depth=5)                           \nclf = clf.fit(sample, classes)\n'

In [7]:
# 支持向量机模型
"""
from sklearn.svm import SVC

clf = SVC(gamma='auto')
clf.fit(sample, classes)
out_fn = 'SVM_prediction60.tif'
"""

"\nfrom sklearn.svm import SVC\n\nclf = SVC(gamma='auto')\nclf.fit(sample, classes)\nout_fn = 'SVM_prediction60.tif'\n"

In [8]:
# 随机森林模型
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, class_weight="balanced")

clf.fit(sample, classes)
out_fn = 'random_forest_prediction60_balanced.tif'

# 预测分类

In [9]:
# Apply model to satellite data
rows, cols, bands = data.shape
data2d = np.reshape(data, (rows * cols, bands)) 
prediction = clf.predict(data2d) 
prediction = np.reshape(prediction, (rows, cols))

# Set pixels with no satellite data to 0。处理图像中的黑色部分
prediction[np.sum(data, 2) == 0] = 0   

predict_ds = pb.make_raster(ds, out_fn, prediction, gdal.GDT_Byte, 0)
predict_ds.FlushCache()
levels = pb.compute_overview_levels(predict_ds.GetRasterBand(1))
predict_ds.BuildOverviews('NEAREST', levels)

# Copy colormap from SWReGAP dataset
gap_ds = gdal.Open(gap_fn)                                             
colors = gap_ds.GetRasterBand(1).GetRasterColorTable()                 
predict_ds.GetRasterBand(1).SetRasterColorTable(colors) 

del ds, predict_ds

该模型要对新图片预测分类的话，必须也要提供四个band,这是机器学习模型的正常要求。

# 版本2：采用HRSCD数据集

## 数据处理：获得训练数据和标签

In [1]:
import numpy as np
from osgeo import gdal

training_raster_fn = r'D:\Download\HRSCD\images_2012\2012\D14\14-2012-0460-6915-LA93-0M50-E080.tif' # 模型训练的数据
label_fn = r"D:\Download\HRSCD\labels_land_cover_2012\2012\D14\14-2012-0460-6915-LA93-0M50-E080.tif" # 训练数据的label

# 将training_raster_fn中的数据读到一个(n, 3)的矩阵中，并且3个band同一个位置处的像素值放在同一行，这个矩阵命名为X
# label_fn是training_raster_fn中像素点的label，即[0,5]之间的6个整数，把它读到一个列表中，用y表示

# 将 training_raster_fn 中的前3个波段读入一个 (n, 3) 的矩阵 X，
# 并将 label_fn 中的像素标签读入一个列表 y（按像素行优先顺序）
ds_train = gdal.Open(training_raster_fn)
if ds_train is None:
    raise IOError(f"无法打开训练影像: {training_raster_fn}")

rows = ds_train.RasterYSize
cols = ds_train.RasterXSize
if ds_train.RasterCount < 3:
    raise ValueError("训练影像波段数少于3，请检查 training_raster_fn")
# 读取前三个波段并堆叠为 (rows, cols, 3)
b1 = ds_train.GetRasterBand(1).ReadAsArray()
b2 = ds_train.GetRasterBand(2).ReadAsArray()
b3 = ds_train.GetRasterBand(3).ReadAsArray()
stack = np.dstack((b1, b2, b3))  # shape = (rows, cols, 3)

# 重塑为 (n, 3)
X = stack.reshape((rows * cols, 3))
# 读取标签栅格并重塑为长度为 n 的列表 y
ds_label = gdal.Open(label_fn)
if ds_label is None:
    raise IOError(f"无法打开标签影像: {label_fn}")

label_arr = ds_label.GetRasterBand(1).ReadAsArray()
if label_arr.shape != (rows, cols):
    raise ValueError("标签影像与训练影像尺寸不匹配")

y = label_arr.reshape(rows * cols).astype(np.uint8).tolist()

# 释放 dataset
ds_train = None
ds_label = None

# 现在 X 是 (n,3) 的 numpy 数组，y 是对应的像素标签列表
print("X.shape =", X.shape, "len(y) =", len(y))

X.shape = (100000000, 3) len(y) = 100000000


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 1000000, test_size=100000, random_state=0)

## 训练模型

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

clf = RandomForestClassifier(n_estimators=100, class_weight="balanced")
clf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [11]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.08      0.76      0.15       638
           1       0.76      0.55      0.63     28662
           2       0.86      0.62      0.72     50415
           3       0.24      0.69      0.35      4242
           4       0.12      0.64      0.20      2167
           5       0.72      0.68      0.70     13876

    accuracy                           0.61    100000
   macro avg       0.46      0.66      0.46    100000
weighted avg       0.76      0.61      0.66    100000



## 保存模型

In [13]:
import joblib
joblib.dump(clf, r".\pkl\random_forest_model.pkl")

['.\\pkl\\random_forest_model.pkl']

## 加载模型

In [ ]:
loaded_model = joblib.load(r".\pkl\random_forest_model.pkl")

## 预测新图片

In [ ]:
new_img_fn = r"D:\Download\HRSCD\images_2012\2012\D14\14-2012-0415-6890-LA93-0M50-E080.tif"

dst_file = r'D:\Projects\VsCode\Python\img_processing_system\tmp\subset_14-2012-0415-6890-LA93-0M50-E080.tif'

# new_img_fn太大了，预测会花费太长时间，因此，裁剪一部分用作预测
ds = gdal.Translate(
    dst_file,
    new_img_fn,
    srcWin=[0, 0, 1000, 1000]  # x_offset, y_offset, width, height
)
del ds


new_img_ds = gdal.Open(dst_file)
rows = new_img_ds.RasterYSize
cols = new_img_ds.RasterXSize
if new_img_ds.RasterCount < 3:
    raise ValueError("训练影像波段数少于3，请检查 new_img_ds")

# 读取前三个波段并堆叠为 (rows, cols, 3)
b1 = new_img_ds.GetRasterBand(1).ReadAsArray()
b2 = new_img_ds.GetRasterBand(2).ReadAsArray()
b3 = new_img_ds.GetRasterBand(3).ReadAsArray()
stack = np.dstack((b1, b2, b3))  # shape = (rows, cols, 3)

# 重塑为 (n, 3)
new_img_X = stack.reshape((rows * cols, 3))

pred_result = clf.predict(new_img_X)
prediction_matrix = np.reshape(pred_result, (rows, cols))
driver = gdal.GetDriverByName('gtiff') 

predict_ds = driver.Create(r"D:\Projects\VsCode\Python\img_processing_system\tmp\prediction_result.tif", cols, rows) 
predict_ds.SetProjection(new_img_ds.GetProjection()) 
predict_ds.SetGeoTransform(new_img_ds.GetGeoTransform())
band = predict_ds.GetRasterBand(1)
band.WriteArray(prediction_matrix)
band.FlushCache() 
del new_img_ds

## 构建colortable

In [21]:
# 构建 colortable，索引对应标签 0-5
ct = gdal.ColorTable()
ct.SetColorEntry(0, (0, 0, 0, 0))         # 0: No information -> 透明
ct.SetColorEntry(1, (255, 0, 0, 255))     # 1: Artificial surfaces -> 红
ct.SetColorEntry(2, (255, 255, 0, 255))   # 2: Agricultural areas -> 黄
ct.SetColorEntry(3, (0, 128, 0, 255))     # 3: Forests -> 绿
ct.SetColorEntry(4, (0, 160, 160, 255))   # 4: Wetlands -> 青/绿蓝
ct.SetColorEntry(5, (0, 0, 255, 255))     # 5: Water -> 蓝

predict_ds.GetRasterBand(1).SetRasterColorTable(ct) 
del predict_ds